# Importing modules

```torch``` - For building the network  
```pickle``` - storing model state  
```mmap``` - Interacting with data from training and validation files  
```random``` - selecting random seek points from data files to collect and train data  
```argparse``` - Optional. Can be used to feed hyper-parameters from  CLI to train/tune the model

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import pickle
import mmap
import random
# import argparse

# parser = argparse.ArgumentParser(description='Something')
# parser.add_argument('--batch-size', type=str, required=True, help='Provide batch size')


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


### Hyper-parameters
```blk_size```  - implying how many characters will be taken into consideration for context to predict the next character. Once the character is predicted, then the block moves by 1 char, and the next 8 charcters (including the last predicted character) is taken into context and so on.  
```batch_size``` - number of blocks that will be stacked together.  
```chkpt``` - Value to check progress at particular number of iterations.  
```max_iter``` - number of times the training loop will run.  
```lr``` - learning rate  
```n_embed``` - length of embedded vector  
```n_EDlayers``` - number of encoder and decoders in the Transformer architecture  
```n_head``` - number of heads for multihead attention  
```dropout``` - percentage of nodes in NN dropping out, for regularization

In [2]:
blk_size = 32
batch_size = 32 # 32 blocks stacked together
chkpt = 100 # Checkpoint for every 100 iterations
max_iter = 1000   # maximum number of iterations
lr = 3e-4              # Learning rate
n_embed = 384     # dimension of embedding vector
n_EDlayers = 4 # number of encoders and decoders in the a
n_head = 4
dropout = 0.2

# Data Handling

We perform character-level tokenization of data. Hence, the vocabulary consists of single characters. This can be extended to word-level tokenization and training.  
We then implement encoding (```enc```) and decoding (```dec```) functions for embedding these characters.



In [3]:
char = ""
with open('vocab.txt', 'r', encoding='utf-8') as f:
    txt = f.read()
    chars = sorted(list(set(txt)))
 
vocab_size = len(chars)

In [4]:
# character-level tokenization

str_to_int = {ch : i for i,ch in enumerate(chars)}
int_to_str = {i : ch for i,ch in enumerate(chars)}
enc = lambda x : [str_to_int[c] for c in x]  # encoding char to int
dec = lambda x : ''.join([int_to_str[i] for i in x])  # decoding int to char

Instead of sequential access of data. We select random chunks of data of size ```blk_size```, and stack them in number of
```batch_size``` blocks.

```x``` and ```y``` contain input data and targets respectively. If data is "hello world", and block size is 3, then if x contains 'ell', y will contain 'llo'.

In [5]:
import mmap
import random
from torch.utils.data import DataLoader, TensorDataset


def get_data(split):
    fname = 'output_train.txt' if split == 'train' else 'output_val.txt'
    with open(fname, 'r') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            fsize = len(mm)     # determine file size and random pos to start reading
            start = random.randint(0, fsize - blk_size*batch_size) # from start to just before the end so that lookahead is possible if required
            
            #seek to that pos and start reading the block of text
            mm.seek(start)
            blk = mm.read(blk_size*batch_size-1)
            
            # decode block to a string and ignoring any invalid byte sequences
            dec_blk = blk.decode('utf-8', errors='ignore').replace('\r','')

            # Train and test split
            data = torch.tensor(enc(dec_blk), dtype=torch.long)
    
    return data


def get_batch(part):
    d = get_data(part)
    idx = torch.randint(len(d) - blk_size, (batch_size,))
    x = torch.stack([d[i:i+blk_size] for i in idx])
    y = torch.stack([d[i+1:i+1+blk_size] for i in idx])
    return x.to(device), y.to(device)


# Initializing Neural Net

We are constructing a GPT of ```n_EDlayers``` number of layers. Each layer will contain a multihead attention component of ```n_head``` heads.

In [6]:
class Head(nn.Module):
    ''' self attention head'''

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(blk_size, blk_size))) # for masking to avoid lookahead
        self.dropout = nn.Dropout(dropout)

    def forward(self, X):
        B, T, C = X.shape
        k = self.key(X) # (B, T, head_size)
        q = self.query(X) # # (B, T, head_size)
        # computing attention score
        wght = q @ k.transpose(-2,-1) * k.shape[-1]**0.5 # Flipping last two dim, and scaling by 1/sq.root
        wght = wght.masked_fill(self.tril[:T, :T] == 0, float('-inf')) 
        wght = F.softmax(wght, dim=-1)
        wght = self.dropout(wght)

        # perform weihted augmentation on values
        v = self.value(X)
        return wght @ v

Each head takes in ```head_size * n_heads``` number of inputs (which is essentially equal to __n_embed__, but we keep it that way
to avoid any computational errors), and produces ```n_embed``` number of outputs.

```nn.ModuleList``` is used instead of ```nn.Sequential``` so that output of each head is not dependent on the output of previous head.
Output of each head is then concatenated.

In [22]:
class MultiHeadAttention(nn.Module):

    def __init__(self, n_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_heads)])
        self.proj = nn.Linear(head_size * n_heads, n_embed) # just to add more learnable parameters
        self.dropout = nn.Dropout(dropout)

    def forward(self, X):
        out = torch.cat([h(X) for h in self.heads], dim=-1) # dim -> (B, T, 4*features)
        out = self.dropout(self.proj(out))

        return out

In [23]:
# Regular MLP

class FeedForward(nn.Module):

    def __init__(self, n_embed):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(n_embed, 4*n_embed),
            nn.ReLU(),
            nn.Linear(4*n_embed, n_embed),
            nn.Dropout(dropout),
        )

    def forward(self, X):
        return self.network(X)

This block represents the encoder and decoder part. Normalization is done after adding the outputs of multi-head components

In [ ]:
class Block(nn.Module):

    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head   # how many features does each head capture
        self.ma = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        y = self.ma(x)
        x = self.ln1(x + y)   # Adding and normalizing
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x    

Every character is vocabulary is encoded/embdedded using nn.Embedding layer which takes in number of characters,
and outputs vectors of size ```n_embed``` for each character in vocabulary.  
For every character that is then considered in a block, we encode them positionally to capture their positional significance.

In [25]:
class GPTLangModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embed = nn.Embedding(vocab_size, n_embed)   # token embedding lookup table
        self.pos_embed_table = nn.Embedding(blk_size, n_embed)   # positional embedding table
        self.layers = nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range(n_EDlayers)])
        self.l_fin = nn.LayerNorm(n_embed)       # Final layer norm
        self.lm_head = nn.Linear(n_embed, vocab_size)    # Final linear layer after encoders-decoders
        self.apply(self._init_weights)

    # Randomly initializing weights using normal distribution
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.2)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.2)
    
    def forward(self, idx, targets=None):
        batch, time = idx.shape
        
        # idx and targets are tensors of shape (batch*time)
        tok_emb = self.token_embed(idx)
        pos_emb = self.pos_embed_table(torch.arange(time, device=device))
        x = tok_emb + pos_emb
        x = self.layers(x)
        x = self.l_fin(x) 
        logits = self.lm_head(x)
        
        
        if targets is None:
            loss = None
        else:
            batch, time, channels = logits.shape       # time is a sequential dimension, channels = vocab size (or) number of classes 
            logits = logits.view(batch*time, channels)   # reshaping logits(B,T,C) dimensions to (B*T, C)
            targets = targets.view(batch*time)    
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    
    # This function generates the characters that the model produces after training
    def generate(self, idx, maxNewTokens):
        for _ in range(maxNewTokens):
            idx_cond = idx[:, -blk_size:]
            logits, loss = self.forward(idx_cond)
            logits = logits[:, -1, :]    # focusing on last timestep as it is a bigram model (single prev char)
            probs = F.softmax(logits, dim = -1)   # get the probabilities of last dimension
            idxNxt = torch.multinomial(probs, num_samples=1) # sample from those probabilities
            idx = torch.cat((idx, idxNxt), dim=1)
        
        return idx

### Loading Pre-trained model

__WARNING__: This should be done only when a pre-trained and saved model is to be loaded and enhanced/fine-tuned by changing hyperparameters.  
Uncomment this code if training for first time

In [ ]:
model = GPTLangModel(vocab_size)

with open('model_01.pkl','rb') as f:
    model = pickle.load(f)
print(' model loaded successfully')
m = model.to(device)

## Loss function and Training

In [12]:
# loss function
@torch.no_grad()

def calc_loss():
    out = {}
    
    model.eval()
    
    for i in ['train', 'val']:
        losses = torch.zeros(chkpt)
        
        for j in range(chkpt):
            X, y = get_batch(i)
            logits, loss = model(X, y)
            losses[j] = loss.item()
        
        out[i] = losses.mean()    
    
    model.train()
    
    return out

In [13]:
# torch.cuda.set_device(0)  # Set to the index of your GPU
# import os
# # os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
# os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [15]:
# Training loop

optim = torch.optim.AdamW(model.parameters(), lr=lr) # defining optimizer

for i in range(max_iter):
    if i % chkpt == 0:
        losses = calc_loss()
        print(f"Epoch: {i} - Train loss: {losses['train']:.4f}, Validation loss: {losses['val']:.4f}")
    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optim.zero_grad(set_to_none=True)
    loss.backward()
    optim.step()

print(loss.item())

Epoch: 0 - Train loss: 2.9732, Validation loss: 2.9260
Epoch: 100 - Train loss: 3.0223, Validation loss: 2.9342
Epoch: 200 - Train loss: 3.0254, Validation loss: 2.9231
Epoch: 300 - Train loss: 2.8425, Validation loss: 3.0704
Epoch: 400 - Train loss: 3.0328, Validation loss: 2.9500
Epoch: 500 - Train loss: 2.9832, Validation loss: 2.9143
Epoch: 600 - Train loss: 3.0034, Validation loss: 2.9441
Epoch: 700 - Train loss: 2.9822, Validation loss: 3.0299
Epoch: 800 - Train loss: 3.0332, Validation loss: 2.9599
Epoch: 900 - Train loss: 2.8665, Validation loss: 2.9640
3.200286626815796


### Saving a Model

Uncomment the code below to save the model with any preferred name.

In [16]:
# with open('model_01.pkl','wb') as f:
#     pickle.dump(model, f)
# print('model saved')

# Testing the model

On Entering the prompt, the model generates a maximum of 100 characters. Exits on prompting "exit".

In [21]:
while True:
    prompt = input("Prompt:\n")
    if prompt == 'exit':
        break
    context = torch.tensor(enc(prompt), dtype=torch.long, device=device)
    generated_chars = dec(m.generate(context.unsqueeze(0), maxNewTokens=100)[0].tolist())
    print(f'Completion:\n{generated_chars}')

Prompt:
 hello


Completion:
hellooept3e(Nsepak ’    cu syom uetspg Da2p n  oto a  3f o4i tat eaist wsdG  e nna s ,t aaipnR
rs emyaegy


Prompt:
 okay


Completion:
okayir se olr noieehaoarroo thy  ih aN n    arsm rent79sc   oeifaoybiaok ideet  gmipnt n0 ,wciaoolklreat


Prompt:
 exit
